In [1]:
import pandas as pd
import numpy as np

In [2]:
events_data = pd.read_csv('data/event_data_train.zip')
submission_data = pd.read_csv('data/submissions_data_train.zip')

submission_data.head()

,step_id,timestamp,submission_status,user_id
0,31971,1434349275,correct,15853
1,31972,1434348300,correct,15853
2,31972,1478852149,wrong,15853
3,31972,1478852164,correct,15853
4,31976,1434348123,wrong,15853


In [13]:
# задача - найти степ-гроб. Степ, после которого наибольшая часть 
# пользователей больше ничего не сабмитила и не пыталась

# 1. выбираем вообще все максимальные времена сабмитов юзеров
df = submission_data.groupby('user_id', as_index=False)\
          .agg({'timestamp':'max'})\
          .rename(columns={'timestamp':'last_timestamp'})
df.head()

,user_id,last_timestamp
0,2,1514383485
1,3,1444573728
2,5,1499859730
3,8,1480604143
4,14,1436368601


In [14]:
# inner - потому что нам нужны только те, у которых есть хоть один степ
df_mix = pd.merge(submission_data, df, on='user_id', how='inner',
                  left_index=False, right_index=False, sort=False,
                  suffixes=('_x', '_y'), copy=True, indicator=False,
                  validate=None)
df_mix_last_wrong = df_mix[ (df_mix['last_timestamp'] == df_mix['timestamp']) 
                          & (df_mix['submission_status'] == 'wrong')]
df_mix_last_wrong

,step_id,timestamp,submission_status,user_id,last_timestamp
1696,33025,1436094981,wrong,18175,1436094981
5588,32173,1435303058,wrong,6053,1435303058
5848,33669,1436431976,wrong,13793,1436431976
6184,33350,1490785375,wrong,19739,1490785375
7332,32202,1492184437,wrong,17578,1492184437
...,...,...,...,...,...
509088,120745,1490136174,wrong,5608,1490136174
509099,120745,1501946959,wrong,2615,1501946959
509101,120745,1506348153,wrong,15253,1506348153
509102,120745,1523035316,wrong,1424,1523035316


In [15]:
df_mix_last_wrong.groupby('step_id', as_index=False)\
    .agg({'submission_status':'count'})\
    .sort_values('submission_status',ascending=False)

,step_id,submission_status
4,31978,154
28,32812,133
11,32031,97
19,32202,92
42,33481,78
...,...,...
43,33482,1
59,33686,1
57,33684,1
55,33676,1
